In [1]:
import numpy as np
#convert isoXXXX into utf8 :
#to use unix command through this notebook, start the command with '!'
#iconv -f iso8859-1 -t utf-8 devTP_SRL.txt > devUTF8.txt

N = 100000 #nb of lines selected in train file 
NLINE_TEST=100#nb of line selected in test file
delta=0.06#Delta used in the second part

train = open("./trainUTF8.txt","r")
test = open("./devUTF8.txt","r")

source =[next(train) for x in range(N)]
source_test = [next(test) for x in range(NLINE_TEST)]

# Cleansing

In [2]:
def rm_ponc( file ):
    punc = '''!()-[]{};:"\, <>./?@#$%^&*_“”‘’~'''
    for ele in file:  
        if ele in punc:  
            file = file.replace(ele, " ")
    return file
#reference = [next(fileTarget) for x in range(N)]
def get_nGram( n, unePhrase ):
    #Remove \n at the end
    unePhrase = rm_ponc(unePhrase).split(" ")
    unePhrase[-1] = unePhrase[-1].rstrip()
    liste = []
    
    for i in range(0, (len(unePhrase)-n+1)):
        ngram = []
        for y in range(0,n):
            ngram.append(unePhrase[i+y])
            
        liste.append(ngram)
        
    return liste

def cTuple2(c):
    return (c[0],c[1])
def cTuple3(c):
    return (c[0],c[1],c[2])
#declaration of Ngrams dictionary
lstBiGram = []
lstTriGram = []
lstUniGram=[]
dicoBiGram={}
dicoUniGram={}
dicoTriGram={}
dicoProbUniGram={}
dicoProbBiGram={}
dicoProbTriGram={}

for c in source :
    lstUniGram.append(get_nGram(1,c))
    lstBiGram.append(get_nGram(2,c))
    lstTriGram.append(get_nGram(3,c))


#unigram
for ph in lstUniGram:
    for tupl in ph:
        if tupl[0] in dicoUniGram:
            dicoUniGram[tupl[0]]+=1
        else:
            dicoUniGram[tupl[0]]=1
#bigram
for ph in lstBiGram:
    for tupl in ph:
        tuples = cTuple2(tupl)
        if tuples in dicoBiGram:
            dicoBiGram[tuples]+=1
        else:
            dicoBiGram[tuples]=1
#trigram
for ph in lstTriGram:
    for tupl in ph:
        tuples = cTuple3(tupl)
        if tuples in dicoTriGram:
            dicoTriGram[tuples]+=1
        else:
            dicoTriGram[tuples]=1

#probability of ngrams, here Bigram
for i in dicoBiGram:
    prob = dicoBiGram[i]/dicoUniGram[i[0]]
    dicoProbBiGram[i]=prob

#probability of ngrams, here Trigram
for i in dicoTriGram:
    prob = dicoTriGram[i]/dicoBiGram[i[0],i[1]]
    dicoProbTriGram[i]=prob

#c(Wi) / c(nb_ mot_tot) ?
#probability of ngrams, here Unigram
for i in dicoUniGram:
    prob = dicoUniGram[i]/sum(dicoUniGram.values())
    dicoProbUniGram[i]=prob

#print(dicoProbUniGram)

# FUNCTIONS

In [3]:
#function which returns the probability of a given Trigram
def probofTrigram(trig, dico_tri, dico_bi):
    return dico_tri[trig] / dico_bi[(trig[0], trig[1])] if trig in dico_tri else 0

#function which returns the probability of a given Bigram
def probofBigram(bigram, dico_bi, dico_uni):
    return dico_bi[bigram] / dico_uni[(bigram[0])] if bigram in dico_bi else 0

#function which returns the probability of a given Unigram
def probofUnigram(unigram, dico_uni):
    return dico_uni[unigram] / len(dico_uni) if unigram in dico_uni else 0

#function giving the probability of each Trigram
def getProbTrigram(corpus, dicoTrigram, dicoBigram,dicFin):
    #splitting in 3grams
    listeTri = []
    for i in corpus:
        listeTri.append(get_nGram(3,i))
    
    for phrases in listeTri:
        for elem in phrases:
            elemTuple = cTuple3(elem)
            if elemTuple not in dicFin:   
                dicFin[elemTuple] = probofTrigram(elemTuple, dicoTrigram,dicoBigram)
            
    return dicFin

#function giving the probability of each Bigram

def getProbBigram(corpus, dicoBigram, dicoUnigram ,dicFin):
    #splitting in 2grams
    listeBi = []
    for i in corpus:
        listeBi.append(get_nGram(2,i))
    
    for phrases in listeBi:
        for elem in phrases:
            elemTuple = cTuple2(elem)
            if elemTuple not in dicFin:   
                dicFin[elemTuple] = probofBigram(elemTuple, dicoBigram,dicoUnigram)
            
    return dicFin

#function giving the probability of each Unigram

def getProbUnigram(corpus,dicoUnigram,dicFin):
    #splitting in 1grams
    listeUni = []
    for i in corpus:
        listeUni.append(get_nGram(1,i))
    
    for phrases in listeUni:
        for elem in phrases:
            if elem[0] not in dicFin:
                dicFin[elem[0]] = probofUnigram(elem[0], dicoUnigram)
            
    return dicFin



# FUNCTION OF THE 2ND PART 

In [16]:
def additive_sm_Trigram(delta, trigram, dicoTrigram, dicoBigram, V):
    return (delta + dicoTrigram[trigram]) / (delta*V + dicoBigram[(trigram[0], trigram[1])]) if trigram in dicoTrigram else (delta) / (delta*V) 

def additive_sm_Bigram(delta, bigram, dicoBigram, dicoUnigram, V):
    return (delta + dicoBigram[bigram]) / (delta*V + dicoUnigram[(bigram[1])])if bigram in dicoBigram else (delta) / (delta*V)

def additive_sm_Unigram(delta, unigram, dicoUnigram, V):
    return (delta + dicoUnigram[unigram]) / (delta*V+V) if unigram in dicoUnigram else (delta) / (delta*V)

def ProbsmTrigram(corpus, dicoTrigram, dicoBigram, V, dicFin, delta):
    #spliting in 3gram
    listeTri = []
    for i in corpus:
        listeTri.append(get_nGram(3,i))
    
    for phrases in listeTri:
        for elem in phrases:
            elemTuple = cTuple3(elem)
            if elemTuple not in dicFin:   
                dicFin[elemTuple] = additive_sm_Trigram(delta, elemTuple, dicoTrigram,dicoBigram, V)
            
    return dicFin

def ProbsmBigram(corpus, dicoBigram, dicoUnigram, V, dicFin, delta):
    #spliting in 2gram
    listeBi = []
    for i in corpus:
        listeBi.append(get_nGram(2,i))
    
    for phrases in listeBi:
        for elem in phrases:
            elemTuple = cTuple2(elem)
            if elemTuple not in dicFin:   
                dicFin[elemTuple] = additive_sm_Bigram(delta, elemTuple, dicoBigram,dicoUnigram, V)
            
    return dicFin


def ProbsmUnigram(corpus, dicoUnigram, V, dicFin, delta):
    #spliting in 1gram
    listeUni = []
    for i in corpus:
        listeUni.append(get_nGram(1,i))
    
    for phrases in listeUni:
        for elem in phrases:
            if elem[0] not in dicFin:   
                dicFin[elem[0]] = additive_sm_Unigram(delta, elem[0],dicoUnigram, V)
            
    return dicFin

def rZipTri(tupl, l):
    return (tupl[0],tupl[1], l)

def ProbUnigram_sm_object(dicoUnigram, V, dicFin, delta):
    for elem in dicoUnigram:
        dicFin[elem] = additive_sm_Unigram(delta, elem,dicoUnigram, V)
    
def ProbBigram_sm_object(userInput, dicoBigram, dicoUnigram, V, dicFin, delta):
    for elem in dicoUnigram:
        if (userInput, elem) in dicoBigram:
            dicFin[elem] = additive_sm_Bigram(delta, (userInput, elem), dicoBigram,dicoUnigram, V)
            
def ProbTrigram_sm_object(bigram , dicoTrigram, dicoBigram,dicoUnigram, V, dicFin, delta):
    for elem in dicoUnigram:
        elemActu = rZipTri(bigram, elem)
        if (elemActu) in dicoTrigram:   
            dicFin[elem] = additive_sm_Trigram(delta, elemActu, dicoTrigram,dicoBigram, V)
            
def lunch():
    userContenu = []
    sentence=' '
    while True:

        print("\nSaisie : ")
        userContenu.append(input())
        if(userContenu[-1] != '0'):
            sentence += userContenu[-1]
            print("\n phrase :")
            print(sentence)

            dicFin = {}

            if len(userContenu) > 1:
                bigram = cTuple2(userContenu[-2:])
                ProbTrigram_sm_object(bigram,dicoTriGram, dicoBiGram, dicoUniGram,len(dicoUniGram), dicFin, delta)
                if(len(dicFin) == 0):
                    ProbBigram_sm_object(userContenu[-1],dicoBiGram, dicoUniGram, len(dicoUniGram), dicFin, delta)
            else:
                ProbBigram_sm_object(userContenu[-1],dicoBiGram, dicoUniGram, len(dicoUniGram), dicFin, delta)

            if(len(dicFin) == 0):
                ProbUnigram_sm_object(dicoUniGram, len(dicoUniGram), dicFin,delta)

            liste_tri = sorted(dicFin.items(), key=lambda x: x[1])
            res = liste_tri[-5:]
            print("\nPropositions :")
            for i in res:
                print(i[0])
            print("\n")

        else:
            print("Fin")
            break

In [5]:

dicFin = {}
rang=source[10:100]
getProbTrigram(rang, dicoTriGram, dicoBiGram,dicFin)



{('en', 'france', 'en'): 0.049019607843137254,
 ('france', 'en', 'tous'): 0.030303030303030304,
 ('en', 'tous', 'cas'): 0.5652173913043478,
 ('ben', 'il', 'y'): 0.5,
 ('il', 'y', 'a'): 0.7812905317769131,
 ('y', 'a', 'moi'): 0.0001466275659824047,
 ('au', 'sursaut', 'démocratique'): 1.0,
 ('sursaut', 'démocratique', 'des'): 1.0,
 ('démocratique', 'des', 'français'): 1.0,
 ('des', 'français', 'doit'): 0.00847457627118644,
 ('français', 'doit', 'répondre'): 1.0,
 ('doit', 'répondre', 'un'): 0.3333333333333333,
 ('répondre', 'un', 'véritable'): 1.0,
 ('un', 'véritable', 'sursaut'): 0.05,
 ('véritable', 'sursaut', 'politique'): 1.0,
 ('je', 'sais', 'plus'): 0.012738853503184714,
 ('sais', 'plus', 'comment'): 0.13333333333333333,
 ('plus', 'comment', 'ça'): 0.1111111111111111,
 ('comment', 'ça', "s'"): 0.2571428571428571,
 ('ça', "s'", 'appelle'): 0.061224489795918366,
 ('merci', "d'", 'avoir'): 0.2857142857142857,
 ("d'", 'avoir', 'été'): 0.0949367088607595,
 ('avoir', 'été', 'là'): 0.0645

In [6]:
dicFin = {}
getProbBigram(rang, dicoBiGram, dicoUniGram,dicFin)


{('en', 'france'): 0.039468592802786015,
 ('france', 'en'): 0.03529411764705882,
 ('en', 'tous'): 0.0014832967883399975,
 ('tous', 'cas'): 0.011514614703277236,
 ('ben', 'il'): 0.041666666666666664,
 ('il', 'y'): 0.2490511184688686,
 ('y', 'a'): 0.6535070908394021,
 ('a', 'moi'): 5.052291214065579e-05,
 ('au', 'sursaut'): 0.00024096385542168674,
 ('sursaut', 'démocratique'): 0.5,
 ('démocratique', 'des'): 0.025,
 ('des', 'français'): 0.01469672437414373,
 ('français', 'doit'): 0.0014925373134328358,
 ('doit', 'répondre'): 0.006396588486140725,
 ('répondre', 'un'): 0.00625,
 ('un', 'véritable'): 0.0015787811809283233,
 ('véritable', 'sursaut'): 0.015384615384615385,
 ('sursaut', 'politique'): 0.5,
 ('je', 'sais'): 0.021724090217240904,
 ('sais', 'plus'): 0.04477611940298507,
 ('plus', 'comment'): 0.0013186813186813187,
 ('comment', 'ça'): 0.09308510638297872,
 ('ça', "s'"): 0.026189203634420097,
 ("s'", 'appelle'): 0.007845503922751962,
 ('merci', "d'"): 0.14,
 ("d'", 'avoir'): 0.014820

# TEST

In [7]:
#Test of 2grams on words which are not present in train
dicBigramTestnotPresent = {}
getProbBigram(source_test, dicoBiGram, dicoUniGram,dicBigramTestnotPresent)



{('un', 'ouvrage'): 0.0002368171771392485,
 ('ouvrage', 'de'): 0.06666666666666667,
 ('de', 'référence'): 0.00010007756010908454,
 ('référence', 'diffusé'): 0,
 ('diffusé', 'à'): 0.7142857142857143,
 ('à', 'plus'): 0.0036669032410693164,
 ('plus', 'de'): 0.18813186813186814,
 ('de', 'cent'): 0.003152443143436163,
 ('cent', 'mille'): 0.0157439446366782,
 ('mille', 'exemplaires'): 0.0008580417580322243,
 ('les', 'violences'): 0.0007815892314372557,
 ('violences', 'ont'): 0.043478260869565216,
 ('ont', 'fait'): 0.05872794800371402,
 ('fait', 'au'): 0.01100187265917603,
 ('au', 'moins'): 0.07301204819277109,
 ('moins', 'sept'): 0.0031269543464665416,
 ('sept', 'blessés'): 0.0033240997229916896,
 ('blessés', 'et'): 0.02,
 ('et', 'causé'): 0,
 ('causé', "d'"): 0,
 ("d'", 'importants'): 0.0002813994934809117,
 ('importants', 'dégâts'): 0.038461538461538464,
 ('dégâts', 'matériels'): 0.15384615384615385,
 ('le', 'reste'): 0.007795257102606245,
 ('reste', 'soit'): 0,
 ('soit', 'quarante'): 0.00

In [8]:
#Test of 3grams on words which are not present in train
dicTrigramTestnotPresent = {}
getProbTrigram(source_test, dicoTriGram, dicoBiGram,dicTrigramTestnotPresent)




{('un', 'ouvrage', 'de'): 0.3333333333333333,
 ('ouvrage', 'de', 'référence'): 1.0,
 ('de', 'référence', 'diffusé'): 0,
 ('référence', 'diffusé', 'à'): 0,
 ('diffusé', 'à', 'plus'): 0,
 ('à', 'plus', 'de'): 0.7741935483870968,
 ('plus', 'de', 'cent'): 0.04750778816199377,
 ('de', 'cent', 'mille'): 0.10317460317460317,
 ('cent', 'mille', 'exemplaires'): 0.02197802197802198,
 ('les', 'violences', 'ont'): 0,
 ('violences', 'ont', 'fait'): 1.0,
 ('ont', 'fait', 'au'): 0.05928853754940711,
 ('fait', 'au', 'moins'): 0.574468085106383,
 ('au', 'moins', 'sept'): 0.009900990099009901,
 ('moins', 'sept', 'blessés'): 0.2,
 ('sept', 'blessés', 'et'): 0,
 ('blessés', 'et', 'causé'): 0,
 ('et', 'causé', "d'"): 0,
 ('causé', "d'", 'importants'): 0,
 ("d'", 'importants', 'dégâts'): 0.3333333333333333,
 ('importants', 'dégâts', 'matériels'): 1.0,
 ('le', 'reste', 'soit'): 0,
 ('reste', 'soit', 'quarante'): 0,
 ('soit', 'quarante', 'millions'): 0.25,
 ('quarante', 'millions', 'est'): 0,
 ('millions', 'e

# 2ND PART

In [9]:
#Test of smoothing on 3grams 

dicTestTri_sm = {}
ProbsmTrigram(rang, dicoTriGram, dicoBiGram,len(dicoUniGram),dicTestTri_sm, delta)


{('en', 'france', 'en'): 0.020586785009861934,
 ('france', 'en', 'tous'): 0.002253434847291503,
 ('en', 'tous', 'cas'): 0.014991505578768539,
 ('ben', 'il', 'y'): 0.001246824127223111,
 ('il', 'y', 'a'): 0.6868514971152312,
 ('y', 'a', 'moi'): 0.00013823394399699538,
 ('au', 'sursaut', 'démocratique'): 0.0012482924301662822,
 ('sursaut', 'démocratique', 'des'): 0.0012482924301662822,
 ('démocratique', 'des', 'français'): 0.0012482924301662822,
 ('des', 'français', 'doit'): 0.0010971267698931856,
 ('français', 'doit', 'répondre'): 0.0012482924301662822,
 ('doit', 'répondre', 'un'): 0.0012453592744019926,
 ('répondre', 'un', 'véritable'): 0.0012482924301662822,
 ('un', 'véritable', 'sursaut'): 0.0012209730925175084,
 ('véritable', 'sursaut', 'politique'): 0.0012482924301662822,
 ('je', 'sais', 'plus'): 0.002049424967169406,
 ('sais', 'plus', 'comment'): 0.004623303270474628,
 ('plus', 'comment', 'ça'): 0.00123664193382799,
 ('comment', 'ça', "s'"): 0.01025861678518049,
 ('ça', "s'", 'app

In [10]:
#Test of smoothing on 2grams 

dicTestBi_sm = {}
ProbsmBigram(rang,dicoBiGram, dicoUniGram,len(dicoUniGram),dicTestBi_sm, delta)


{('en', 'france'): 0.2251743826706301,
 ('france', 'en'): 0.0040393392262274555,
 ('en', 'tous'): 0.01166319367173117,
 ('tous', 'cas'): 0.004891092668604129,
 ('ben', 'il'): 8.042426532824032e-05,
 ('il', 'y'): 0.5466122422936223,
 ('y', 'a'): 0.33041069397262557,
 ('a', 'moi'): 0.0006112469437652813,
 ('au', 'sursaut'): 0.001246824127223111,
 ('sursaut', 'démocratique'): 0.001193478652495046,
 ('démocratique', 'des'): 0.00011940755827314142,
 ('des', 'français'): 0.07776518943984824,
 ('français', 'doit'): 0.0008047617601506272,
 ('doit', 'répondre'): 0.0030352325027773372,
 ('répondre', 'un'): 7.842464131824424e-05,
 ('un', 'véritable'): 0.02196767269700819,
 ('véritable', 'sursaut'): 0.001246824127223111,
 ('sursaut', 'politique'): 0.0005911352026589931,
 ('je', 'sais'): 0.10345418137745693,
 ('sais', 'plus'): 0.003917551569366467,
 ('plus', 'comment'): 0.00740099333420468,
 ('comment', 'ça'): 0.012893687756513042,
 ('ça', "s'"): 0.008430770076780841,
 ("s'", 'appelle'): 0.04034457

# Test on not knowed ngrams with Smoothing

In [11]:
#Test of 3grams not present in Train with smoothing
dicTrigramTestnotPresent = {}
ProbsmTrigram(source_test, dicoTriGram, dicoBiGram,len(dicoUniGram),dicTrigramTestnotPresent, delta)


{('un', 'ouvrage', 'de'): 0.0012453592744019926,
 ('ouvrage', 'de', 'référence'): 0.0012482924301662822,
 ('de', 'référence', 'diffusé'): 7.074136955291455e-05,
 ('référence', 'diffusé', 'à'): 7.074136955291455e-05,
 ('diffusé', 'à', 'plus'): 7.074136955291455e-05,
 ('à', 'plus', 'de'): 0.05280390261053002,
 ('plus', 'de', 'cent'): 0.028637625694131776,
 ('de', 'cent', 'mille'): 0.01340642194300731,
 ('cent', 'mille', 'exemplaires'): 0.0039411353576143515,
 ('les', 'violences', 'ont'): 7.074136955291455e-05,
 ('violences', 'ont', 'fait'): 0.0012482924301662822,
 ('ont', 'fait', 'au'): 0.013676486614116025,
 ('fait', 'au', 'moins'): 0.030229232762858035,
 ('au', 'moins', 'sept'): 0.0026581882622745755,
 ('moins', 'sept', 'blessés'): 0.0012424398705987154,
 ('sept', 'blessés', 'et'): 7.074136955291455e-05,
 ('blessés', 'et', 'causé'): 7.074136955291455e-05,
 ('et', 'causé', "d'"): 7.074136955291455e-05,
 ('causé', "d'", 'importants'): 7.074136955291455e-05,
 ("d'", 'importants', 'dégâts'

In [12]:
#Test of 2grams not present in Train with smoothing

dicBigramTestnotPresent = {}
ProbsmBigram(source_test,dicoBiGram, dicoUniGram,len(dicoUniGram),dicBigramTestnotPresent, delta)


{('un', 'ouvrage'): 0.003545113304601696,
 ('ouvrage', 'de'): 2.5969469703428656e-05,
 ('de', 'référence'): 0.004698204036289576,
 ('référence', 'diffusé'): 7.074136955291455e-05,
 ('diffusé', 'à'): 0.0002849715253748558,
 ('à', 'plus'): 0.008087932481533033,
 ('plus', 'de'): 0.03145882761074018,
 ('de', 'cent'): 0.010159443463011437,
 ('cent', 'mille'): 0.01605869547576289,
 ('mille', 'exemplaires'): 0.010520693018718939,
 ('les', 'violences'): 0.010399926534735296,
 ('violences', 'ont'): 0.0002055793458697946,
 ('ont', 'fait'): 0.049424236742601796,
 ('fait', 'au'): 0.009415464891079918,
 ('au', 'moins'): 0.1238415142450841,
 ('moins', 'sept'): 0.0019071597642057018,
 ('sept', 'blessés'): 0.005278010033444816,
 ('blessés', 'et'): 0.0004516604109960677,
 ('et', 'causé'): 7.074136955291455e-05,
 ('causé', "d'"): 7.074136955291455e-05,
 ("d'", 'importants'): 0.003500503340349593,
 ('importants', 'dégâts'): 0.0011948239325488075,
 ('dégâts', 'matériels'): 0.007045200892857143,
 ('le', 'r

# Predict the next word in 5 output using ngram probabilities

In [15]:
#5 most probable words, we first search in 3grams, if not then 2grams, if not then 1gram
lunch()



Saisie : 
je

la phrase :
 je

Propositions :
vais
me
crois
pense
suis



Saisie : 
vais

la phrase :
 je vais

Propositions :
à
le
essayer
faire
vous



Saisie : 
faire

la phrase :
 je vais faire

Propositions :
partie
pendant
ensuite
maintenant
une



Saisie : 
une

la phrase :
 je vais faire une

Propositions :
pause
bonne
idée
grande
mise



Saisie : 
pause

la phrase :
 je vais faire une pause

Propositions :
comme
sur
dans



Saisie : 
dans

la phrase :
 je vais faire une pause dans

Propositions :
la
le



Saisie : 
la

la phrase :
 je vais faire une pause dans la

Propositions :
nuit
tête
région
rue
vie



Saisie : 
nuit

la phrase :
 je vais faire une pause dans la nuit

Propositions :
est
des
a
du
de



Saisie : 
0
Fin
